In [2]:
import pandas as pd
from lime.lime_text import LimeTextExplainer
from joblib import Parallel, delayed
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import gensim
from gensim.models import Word2Vec
import gensim.downloader as api
from gensim.models import KeyedVectors
from gensim.parsing.preprocessing import strip_punctuation
print((gensim.__version__)) 
import pickle
import tqdm
from tqdm import tqdm
from typing import Tuple, List
from scipy.stats import entropy

4.1.2


In [3]:
test_df = pd.read_csv("C:/Users/berl03/Desktop/LIME/1 příprava dat/y_test.csv")
model_w2v=gensim.models.Word2Vec.load("C:/Users/berl03/Desktop/LIME/1 příprava dat/w2v_published_only_english_between_2019 and 2020.model")
loaded_model = lreg_w2v_avg = pickle.load(open("C:/Users/berl03/Desktop/LIME/1 příprava dat/lreg_w2v_avg_2020.sav", 'rb'))

# LIME applied on test_df

In [4]:
def predict_probabilities_from_text(texts: List[str]) -> np.ndarray:
    """
    Converts a list of texts into feature vectors using a Word2Vec model 
    and predicts probabilities using LR model.
    Parameters:
    - texts (list of str): List of textual data.
    Returns:
    - ndarray: Probabilities predicted by the model for each text.
    """
    vectors = []
    # Convert each text into a feature vector
    for text in texts:
        # Tokenize the text and filter words present in the Word2Vec model
        words = [word for word in text.split() if word in model_w2v.wv]
        # If there are valid words, compute the mean vector
        if words:
            vector = np.mean([model_w2v.wv[word] for word in words], axis=0)
            vectors.append(vector)
        else:
            # If no valid words, append a zero vector
            vectors.append(np.zeros(model_w2v.vector_size))
    # Predict probabilities using the preloaded model
    return loaded_model.predict_proba(vectors)

In [5]:
def process_instance(i, text, true_label, word_cache):
    explained = explainer.explain_instance(text, predict_probabilities_from_text, num_features=15)
    lime_word_importances = explained.as_list()
    importances_df = pd.DataFrame(lime_word_importances, columns=["word", "importance"])
    importances_df["true_label"] = true_label
    importances_df["abstract_text"] = text
    importances_df["intercept"] = explained.intercept[1]
    importances_df["prediction_local"] = explained.local_pred[0]

    for word in importances_df["word"].unique():
        if word not in word_cache:
            word_cache[word] = {}
            word_cache[word]["predicted_proba"] = predict_probabilities_from_text([word])[0][1]
            word_cache[word]["cnt_of_articles_with_targ"] = (test_df["abstract_cleaned"].str.contains(word) & test_df["target"].notna()).sum()
            word_cache[word]["%_of_high_cit"] = (test_df["abstract_cleaned"].str.contains(word) & (test_df["target"] == 1)).sum() / word_cache[word]["cnt_of_articles_with_targ"]

        importances_df.loc[importances_df["word"] == word, "predicted_proba"] = word_cache[word]["predicted_proba"]
        importances_df.loc[importances_df["word"] == word, "cnt_of_articles_with_targ"] = word_cache[word]["cnt_of_articles_with_targ"]
        importances_df.loc[importances_df["word"] == word, "%_of_high_cit"] = word_cache[word]["%_of_high_cit"]

    return importances_df

In [6]:
explainer = LimeTextExplainer(class_names=["Low", "High"], random_state=42)
word_cache = {}
results = Parallel(n_jobs=-1)(delayed(process_instance)(i, text, true_label, word_cache) for i, (text, true_label) in enumerate(tqdm(zip(test_df["abstract_cleaned"], test_df["target"]))))
lime_df = pd.concat(results).reset_index(drop=True)

8645it [1:07:58,  2.12it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'outputs/lime/lime_data_correct.csv'

In [7]:
lime_df.to_csv("lime_data_correct.csv")

In [18]:
def calculate_word_metrics(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculates various metrics related to word importance, such as the number of occurrences,
    positive importances, and average importances for high and low citation articles.
    Parameters:
    - df (pd.DataFrame): DataFrame containing word importances and true labels.
    Returns:
    - pd.DataFrame: DataFrame with new columns containing the calculated metrics.
    """
    
    df['importance'] = df['importance'] + 1 / 2
    
    # Calculate absolute importance
    df['abs_importance'] = df['importance'].abs()

    # Global LIME Importance (square root of the sum of absolute importance)
    global_lime_importance = np.sqrt(df.groupby('word')['abs_importance'].sum())

    # Global Average Importance
    count_nonzero_importance = df.groupby('word')['abs_importance'].apply(lambda x: (x != 0).sum())
    global_avg_importance = df.groupby('word')['abs_importance'].sum() / count_nonzero_importance

    # Normalize LIME importance per class
    df['normalized_importance'] = df.groupby(['word', 'true_label'])['abs_importance'].transform(lambda x: x / x.sum())

    # Calculate the entropy values for each word based on normalized importance
    grouped = df.groupby(['word', 'true_label'])['normalized_importance'].mean().reset_index()
    grouped = grouped.pivot(index='word', columns='true_label', values='normalized_importance').fillna(0)
    
    # Calculate Shannon entropy for the distribution of normalized importance over classes
    entropy_values = grouped.apply(lambda x: entropy(x + 1e-9), axis=1)

    # Normalize the entropy values
    min_entropy = entropy_values.min()
    max_entropy = entropy_values.max()
    entropy_values_normalized = (entropy_values - min_entropy) / (max_entropy - min_entropy)

    # Calculate the Global Homogeneity-Weighted Importance for each word
    global_homogeneity_importance = (1 - entropy_values_normalized) * global_lime_importance


    total_occurrences = df.groupby('word').size()
    avg_predict_proba = df.groupby('word')['predicted_proba'].mean()

    # Map the calculated metrics back to the original dataframe
    metrics_df = df.assign(
        total_occurrences=df['word'].map(total_occurrences),
        avg_predict_proba=df['word'].map(avg_predict_proba),
        global_lime_importance=df['word'].map(global_lime_importance),
        global_avg_importance= df["word"].map(global_avg_importance),
        global_homogeneity_importance=df['word'].map(global_homogeneity_importance),
    )

    metrics_df.fillna(0, inplace=True)
    return metrics_df

In [19]:
lime_df_word_metrics = calculate_word_metrics(lime_df)

In [20]:
df_word_metrics_no_duplicates = lime_df_word_metrics.drop_duplicates(subset='word', keep='first')

In [21]:
lime_df_word_metrics.drop_duplicates(subset='word', keep='first')[["word","global_lime_importance","global_avg_importance"]].to_csv("lime_df_fin_v2.csv")

In [22]:
df_word_metrics_no_duplicates

word  importance  true_label  \
0             icu    0.572772           0   
1       resistant    0.466021           0   
2             mdr    0.468001           0   
3             sub    0.475227           0   
4             amr    0.480435           0   
...           ...         ...         ...   
129360  bolstered    0.485519           1   
129362  draconian    0.512929           1   
129364  countless    0.512062           1   
129365     remedy    0.489494           1   
129374     atpase    0.477689           0   

                                            abstract_text  intercept  \
0       antimicrobial resistance amr compromise treatm...   0.415438   
1       antimicrobial resistance amr compromise treatm...   0.415438   
2       antimicrobial resistance amr compromise treatm...   0.415438   
3       antimicrobial resistance amr compromise treatm...   0.415438   
4       antimicrobial resistance amr compromise treatm...   0.415438   
...                                                   ...        ...   
129360  detrimental impact recreational drug use human...   0.271538   
129362  detrimental impact recreational drug use human...   0.271538   
129364  detrimental impact recreational drug use human...   0.271538   
129365  detrimental impact recreational drug use human...   0.271538   
129374  reemergence coronavirus prompt need developmen...   0.638224   

        prediction_local  predicted_proba  cnt_of_articles_with_targ  \
0               0.311023         0.984267                     1466.0   
1               0.311023         0.136985                       70.0   
2               0.311023         0.017721                        7.0   
3               0.311023         0.049098                     1541.0   
4               0.311023         0.093068                        5.0   
...                  ...              ...                        ...   
129360          0.258160         0.074457                        1.0   
129362          0.258160         0.580577                        3.0   
129364          0.258160         0.585605                        7.0   
129365          0.258160         0.119841                       17.0   
129374          0.655212         0.153065                        3.0   

        %_of_high_cit  abs_importance  normalized_importance  \
0            0.379263        0.572772               0.008145   
1            0.257143        0.466021               0.138796   
2            0.142857        0.468001               0.488037   
3            0.369890        0.475227               0.089893   
4            0.200000        0.480435               0.520692   
...               ...             ...                    ...   
129360       1.000000        0.485519               1.000000   
129362       0.333333        0.512929               1.000000   
129364       0.428571        0.512062               1.000000   
129365       0.176471        0.489494               1.000000   
129374       0.333333        0.477689               1.000000   

        total_occurrences  avg_predict_proba  global_lime_importance  \
0                     225           0.984267               10.904456   
1                      10           0.136985                2.191641   
2                       3           0.017721                1.198118   
3                      17           0.049098                2.858660   
4                       3           0.093068                1.187353   
...                   ...                ...                     ...   
129360                  1           0.074457                0.696792   
129362                  1           0.580577                0.716191   
129364                  1           0.585605                0.715585   
129365                  1           0.119841                0.699639   
129374                  1           0.153065                0.691151   

        global_avg_importance  global_homogeneity_importance  
0                    0.528476                